In [1]:
import firebase_admin
import pandas as pd
from firebase_admin import credentials, firestore
cred = credentials.Certificate("store.json")
firebase_admin.initialize_app(cred)


In [2]:
import random

In [10]:
db = firestore.client()
product_ref = db.collection('Products1')
transactions_ref = db.collection('Transactions1')

In [81]:
file = pd.read_csv('https://firebasestorage.googleapis.com/v0/b/online-gift-store-bb0d8.appspot.com/o/online%20retail%20clean%20small.csv?alt=media&token=e4e21322-92d6-4d1b-8fba-eb8ff6394396')

In [82]:
product_cols = file[['StockCode','Name','Description','Quantity','UnitPrice','Picture']]

In [83]:
def writeData(row):
     stockCode = row[0]
     name = row[1]
     description = row[2]
     quantity = row[3]
     unitPrice = row[4]
     picture = row[5]
     searchQueries = name.split()

     data = {'StockCode': stockCode, 'Name': name, 'Description': description, 'Quantity': quantity,
             'UnitPrice': unitPrice, 'Picture': picture, 'QuantitySold': 0, 'Ratings Sum': 0,
             'Ratings Count': 0, 'Color': '', 'SearchQueries': searchQueries}

     product_ref.document(stockCode).set(data)

     return



In [84]:
product_cols.apply(writeData , axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
144    None
145    None
146    None
147    None
148    None
Length: 149, dtype: object

In [85]:
invoicesVisited = {}
invoicesVisitedList = []

In [86]:
transactions_cols = file[['StockCode','InvoiceDate','InvoiceNo','CustomerID']]

In [87]:
import datetime

invoicesVisited = {}
invoicesVisitedList = []

for j in transactions_cols.iterrows():
    i = j[1]

    invoiceNo = i['InvoiceNo']
    invoiceDate = i['InvoiceDate']
    stockCode = str(i['StockCode'])
    customerID = i['CustomerID']

    quantity = 1
    try:
        quantity = int(transactions_ref.document(stockCode).get().to_dict()['QuantitySold']) + 1
        product_ref.document(str(i['StockCode'])).update({'QuantitySold' : quantity})
    except:
        product_ref.document(str(i['StockCode'])).set({'QuantitySold' : quantity})

    if invoiceNo not in invoicesVisitedList:

        invoicesVisitedList.append(invoiceNo)
        invoicesVisited[invoiceNo] = [stockCode]

        d = {}
        d['StockCode'] = [stockCode]

        element = datetime.datetime.strptime(invoiceDate,"%d-%m-%Y %H:%M")
        x = element.timetuple()
        y = str('0' + str(x.tm_year) if len(str(x.tm_year)) == 1 else str(x.tm_year)) + \
            str('0' + str(x.tm_mon) if len(str(x.tm_mon)) == 1 else str(x.tm_mon)) + \
            str('0' + str(x.tm_mday) if len(str(x.tm_mday)) == 1 else str(x.tm_mday)) + \
            str('0' + str(x.tm_hour) if len(str(x.tm_hour)) == 1 else str(x.tm_hour)) + \
            str('0' + str(x.tm_min) if len(str(x.tm_min)) == 1 else str(x.tm_min))

        d['InvoiceDate'] =  int(y)
        d['CustomerID'] = i['CustomerID']
        d['OrderDelivered'] = random.choice([True, False])

        transactions_ref.document(str(invoiceNo)).set(d)


    else:

        l = invoicesVisited[invoiceNo]

        l.append(str(i['StockCode']))
        invoicesVisited[invoiceNo] = l

        transactions_ref.document(str(invoiceNo)).update({'StockCode' : l})


